In [ ]:
!pip install google-cloud-bigquery-storage>=2.30.0,<3.0.0 scipy<1.14.0,>=1.7.0 numpy==1.26.4 pyarrow>=21.0.0 protobuf>=4.25.1 google-auth==2.38.0 notebook==6.5.7 pandas==2.2.2 requests==2.32.3 tornado==6.4.2 gymnasium>=1.0.0 google-cloud-bigquery[bqstorage,pandas]>=3.31.0,<4 rich>=12.4.4,<14 scikit-learn>=1.3.2,<2 pydantic<2.12,>=2.0 google-api-core>=2.10.2,<3 huggingface-hub>=0.30.0,<1 matplotlib>=3.8.0 pylibraft-cu12==25.6.* rmm-cu12==25.6.* ipywidgets scikit-learn==1.3.2 mlxtend>=0.23.4